In [3]:
import os
import sys
 
spark_path = 'C:\Spark\spark-2.1.0-bin-hadoop2.7'
 
os.environ['SPARK_HOME'] = spark_path
os.environ['HADOOP_HOME'] = spark_path
 
sys.path.append(spark_path + "/bin")
sys.path.append(spark_path + "/python")
sys.path.append(spark_path + "/python/pyspark/")
sys.path.append(spark_path + "/python/lib")
sys.path.append(spark_path + "/python/lib/pyspark.zip")
sys.path.append(spark_path + "/python/lib/py4j-0.10.4-src.zip")
 
from pyspark import SparkContext
from pyspark import SparkConf
 
sc = SparkContext("local", "test")
sc

In [3]:
#sc.stop()

In [5]:
ratings_raw_data = sc.textFile("ml-small/ratings.csv")
ratings_raw_data_header = ratings_raw_data.take(1)[0]

In [6]:
ratings_raw_data_header

'userId,movieId,rating,timestamp'

In [7]:
ratings_data = ratings_raw_data.filter(lambda line: line!=ratings_raw_data_header)\
    .map(lambda line: line.split(",")).map(lambda tokens: (tokens[0],tokens[1],tokens[2])).cache()

In [8]:
ratings_1_data_forM_U = ratings_raw_data.filter(lambda line: line!=ratings_raw_data_header)\
    .map(lambda line: line.split(",")).map(lambda tokens: (tokens[1],tokens[0],tokens[2])).cache()

In [9]:
ratings_data.take(3)

[('1', '31', '2.5'), ('1', '1029', '3.0'), ('1', '1061', '3.0')]

In [10]:
movies_raw_data = sc.textFile("ml-small/movies.csv")
movies_raw_data_header = movies_raw_data.take(1)[0]

movies_data = movies_raw_data.filter(lambda line: line!=movies_raw_data_header)\
    .map(lambda line: line.split(",")).map(lambda tokens: (tokens[0],tokens[1])).cache()
    
movies_data.take(3)

[('1', 'Toy Story (1995)'),
 ('2', 'Jumanji (1995)'),
 ('3', 'Grumpier Old Men (1995)')]

In [11]:
##For user-movies-ratings tuple
#training_RDD=ratings_data
training_RDD, test_RDD = ratings_data.randomSplit([10,0], seed=9)
#validation = validation_RDD.map(lambda x: (x[0], x[1],x[2]))
#test = test_RDD.map(lambda x: (x[0], x[1],x[2]))

In [12]:
##For Movies-user-ratings tuple
#traning_RDD1=ratings_1_data_forM_U
training_RDD1,test_RDD1 = ratings_1_data_forM_U.randomSplit([10,0], seed=9)
#validation1 = validation_RDD1.map(lambda x: (x[0], x[1],x[2]))
test1 = test_RDD1.map(lambda x: (x[0], x[1],x[2]))

In [13]:
# Creating sparse representation of A matrix with users as rows and items as columns
user_item_ratings = (training_RDD
                          .map(lambda p: (p[0],p[-2:])).groupByKey()).cache()
#user_item_ratings = ratings_data.map(lambda l: (l[0], l[1],l[2])).cache()
#user_item_ratings = user_item_ratings.groupByKey()

#user_item_ratings_test = (test_RDD
#                          .map(lambda p: (p[0],p[-2:])).groupByKey()).cache()

In [14]:
user_item_ratings.take(5)


[('190', <pyspark.resultiterable.ResultIterable at 0x1bbdf6f7048>),
 ('18', <pyspark.resultiterable.ResultIterable at 0x1bbdf6f8b38>),
 ('17', <pyspark.resultiterable.ResultIterable at 0x1bbdf6fa208>),
 ('142', <pyspark.resultiterable.ResultIterable at 0x1bbdf6f8b70>),
 ('110', <pyspark.resultiterable.ResultIterable at 0x1bbdf705eb8>)]

In [15]:
# Creating sparse representation of transposed A matrix with items as rows and rows as columns
item_user_ratings=(training_RDD1
                          .map(lambda p: (p[0],p[-2:])).groupByKey()).cache()

#item_user_ratings = ratings_data.map(lambda l: (l[0], l[1]))
#item_user_ratings = item_user_ratings.groupByKey()
#item_user_ratings_test=(test_RDD1
 #                         .map(lambda p: (p[0],p[-2:])).groupByKey()).cache()

In [16]:
user_item_ratings.count()

671

In [17]:
item_user_ratings.take(5)

[('90647', <pyspark.resultiterable.ResultIterable at 0x1bbdf594518>),
 ('7560', <pyspark.resultiterable.ResultIterable at 0x1bbdf7095f8>),
 ('38061', <pyspark.resultiterable.ResultIterable at 0x1bbdf7094a8>),
 ('66798', <pyspark.resultiterable.ResultIterable at 0x1bbdf709588>),
 ('94919', <pyspark.resultiterable.ResultIterable at 0x1bbdf710550>)]

In [18]:
item_user_ratings.count()
#user_item_ratings.count()

9066

In [19]:
import numpy as np

In [20]:
# User Defined Parameters
lambda_ = sc.broadcast(0.1) # Regularization parameter
n_factors = sc.broadcast(3) # nfactors of User matrix and Item matrix
n_iterations = 20 # How many times to iterate over the user and item matrix calculations.

In [21]:
# Initizializing Items Matrix (User matrix doesn't need to be initialized since it is solved for first):
Items=dict()
Items = item_user_ratings.map(lambda line: (line[0], 5 * np.random.rand(1, n_factors.value)))

In [22]:
Items.take(10)


[('90647', array([[ 3.59754351,  4.57465601,  1.51621733]])),
 ('7560', array([[ 0.29501107,  1.15549485,  4.86908725]])),
 ('38061', array([[ 3.1471971 ,  2.30878322,  4.24195064]])),
 ('66798', array([[ 0.61408862,  1.11966706,  1.10136137]])),
 ('94919', array([[ 1.77962414,  1.99208691,  0.48839487]])),
 ('66934', array([[ 3.55976329,  4.09466902,  2.41651128]])),
 ('2265', array([[ 3.55370045,  0.84058546,  0.79886226]])),
 ('118702', array([[ 0.02688981,  0.21764501,  3.61974039]])),
 ('3445', array([[ 2.18276652,  2.84225791,  3.67727637]])),
 ('3076', array([[ 1.65687252,  0.76279336,  0.88217267]]))]

In [23]:
# The item matrix is needed in all partitions when solving for rows of User matrix individually
Items_broadcast = sc.broadcast({
  k: v for (k, v) in Items.collect()
})

In [24]:
from pandas import *
from itertools import *
j = 0
for k, v in {k: v for (k, v) in Items.collect()}.items():
    print(k, v)
    j+=1
    if j > 10:
        break

85131 [[ 2.56705075  1.41453574  3.07652181]]
98795 [[ 1.33924273  0.41807334  3.12963595]]
3422 [[ 1.76897853  0.10198196  3.52148978]]
6493 [[ 2.0320321   0.67894347  4.93358818]]
128606 [[ 3.7969534   1.17686204  3.96504826]]
2280 [[ 0.81348073  4.17505485  3.4302541 ]]
4464 [[ 2.55140231  1.98768295  0.39889021]]
2836 [[ 2.98542211  2.01685309  1.63363825]]
4381 [[ 0.50091287  1.96054318  4.5235805 ]]
6260 [[ 1.27226857  0.22989539  0.1033351 ]]
2168 [[ 2.7697963   3.26529862  2.43409945]]


In [25]:

j = 0
for i in user_item_ratings.take(1)[0][1]:
    #print(i)
    j+=1
    if j > 10:
        break

In [26]:
user_item_ratings.take(10)

[('190', <pyspark.resultiterable.ResultIterable at 0x1bbe04e54a8>),
 ('18', <pyspark.resultiterable.ResultIterable at 0x1bbe0484e10>),
 ('17', <pyspark.resultiterable.ResultIterable at 0x1bbe0488710>),
 ('142', <pyspark.resultiterable.ResultIterable at 0x1bbe04841d0>),
 ('110', <pyspark.resultiterable.ResultIterable at 0x1bbe04e1c88>),
 ('223', <pyspark.resultiterable.ResultIterable at 0x1bbe04d4208>),
 ('406', <pyspark.resultiterable.ResultIterable at 0x1bbe04b75f8>),
 ('473', <pyspark.resultiterable.ResultIterable at 0x1bbe05012e8>),
 ('198', <pyspark.resultiterable.ResultIterable at 0x1bbe0466898>),
 ('169', <pyspark.resultiterable.ResultIterable at 0x1bbe046ba58>)]

In [27]:
user_item_ratings.first()

('190', <pyspark.resultiterable.ResultIterable at 0x1bbe046e470>)

In [28]:

#######ALS User matrix row calculation
#Users[i] = inverse(Items*Items^T + I*lambda) * Items * A[i]^T
#Where only rows of item matrix for which user i has ratings are operated upon.

def Update_User(userTuple):
    '''
    This function calculates (userID, Users[i]) using:
        'Users[i] = inverse(Items*Items^T + I*lambda) * Items * A[i]^T'
    Dot product calculations are done differently than normal to allow for sparsity. Rather 
    than row of left matrix times column of right matrix, sum result of column of left matrix  
    * rows of right matrix (skipping items for which user doesn't have a rating).
    '''
    Itemssquare = np.zeros([n_factors.value,n_factors.value])
    for matrixA_item_Tuple in userTuple[1]:
        itemRow = Items_broadcast.value[matrixA_item_Tuple[0]][0]
        for i in range(n_factors.value):
            for j in range(n_factors.value):
                Itemssquare[i,j] += float(itemRow[i]) * float(itemRow[j])
    leftMatrix = np.linalg.inv(Itemssquare + lambda_.value * np.eye(n_factors.value))
    rightMatrix = np.zeros([1,n_factors.value])
    for matrixA_item_Tuple in userTuple[1]:
        for i in range(n_factors.value):
            rightMatrix[0][i] += Items_broadcast.value[matrixA_item_Tuple[0]][0][i] * float(matrixA_item_Tuple[1])
    newUserRow = np.dot(leftMatrix, rightMatrix.T).T
    return (userTuple[0], newUserRow)

In [29]:
Users = user_item_ratings.map(Update_User)


In [30]:
Users.take(3)

[('190', array([[ 0.45807759,  0.43493554,  0.52296561]])),
 ('18', array([[ 0.37875541,  0.41822686,  0.36824639]])),
 ('17', array([[ 0.41871234,  0.39423321,  0.50284481]]))]

In [31]:
#######ALS Item matrix row calculation
#Items[i] = inverse(Users*Users^T + I*lambda) * Users * A[i]^T
#Where only rows of item matrix for which user i has ratings are operated upon.

def Update_Item(userTuple):
    '''
    This function calculates (userID, Items[i]) using:
        'Items[i] = inverse(Users*Users^T + I*lambda) * Users * A[i]^T'
    Dot product calculations are done differently than normal to allow for sparsity. Rather 
    than row of left matrix times column of right matrix, sum result of column of left matrix  
    * rows of right matrix (skipping items for which user doesn't have a rating).
    '''
    Userssquare = np.zeros([n_factors.value,n_factors.value])
    for matrixA_user_Tuple in userTuple[1]:
        userRow = Users_broadcast.value[matrixA_user_Tuple[0]][0]
        for i in range(n_factors.value):
            for j in range(n_factors.value):
                Userssquare[i,j] += float(userRow[i]) * float(userRow[j])
    leftMatrix = np.linalg.inv(Userssquare + lambda_.value * np.eye(n_factors.value))
    rightMatrix = np.zeros([1,n_factors.value])
    for matrixA_user_Tuple in userTuple[1]:
        for i in range(n_factors.value):
            rightMatrix[0][i] += Users_broadcast.value[matrixA_user_Tuple[0]][0][i] * float(matrixA_user_Tuple[1])
    newItemRow = np.dot(leftMatrix, rightMatrix.T).T
    return (userTuple[0], newItemRow)

In [32]:

# The item matrix is needed in all partitions when solving for rows of User matrix individually
Users_broadcast = sc.broadcast({
  k: v for (k, v) in Users.collect()
})

In [33]:
# The item matrix is needed in all partitions when solving for rows of User matrix individually
Items_broadcast = sc.broadcast({
  k: v for (k, v) in Items.collect()
})

In [34]:
Users.first()

('190', array([[ 0.36191441,  0.59740863,  0.39422331]]))

In [35]:
def getRowSumSquares(userTuple):
    userRow = Users_broadcast.value[userTuple[0]]
    rowSSE = 0.0
    for matrixA_item_Tuple in userTuple[1]:
        predictedRating = 0.0
        for i in range(n_factors.value):
            predictedRating += userRow[0][i] * Items_broadcast.value[matrixA_item_Tuple[0]][0][i]
        SE = (float(matrixA_item_Tuple[1]) - predictedRating) ** 2
        rowSSE += SE
    return rowSSE

In [36]:

SSE = user_item_ratings.map(getRowSumSquares).reduce(lambda a, b: a + b)
Count = ratings_data.count()
MSE = SSE / Count
print ("MSE:", MSE)

MSE: 2.21890270294


In [37]:
Items = item_user_ratings.map(Update_Item)


In [38]:
Items.take(3)

[('90647', array([[ 2.77863398,  2.37642494,  1.34226666]])),
 ('7560', array([[ 2.37907809,  2.72821329,  2.8195059 ]])),
 ('38061', array([[ 3.64841424,  3.95474206,  1.25192964]]))]

In [39]:
import pyspark
from pyspark import SparkContext
from pyspark.sql import SQLContext
sql_context = SQLContext(sc)

In [40]:
# Items Transpose
Users.cartesian(Items)\
.filter(lambda x: x [0][0] == x[1][1] and x[0][1] == x[1][0])\
.map(lambda x: (x[0][0],x[0][1] * x[1][2])).reduceByKey(lambda x,y: x+y).collect()#domainslist.take(2) # still returns everything

[]

In [41]:
Users_np_array= Users.map(lambda x: (x[1]))

#Items_np_array=Items.map(lambda x: (x[0], tuple(np.array(_) for _ in zip(*x[1:]))))

In [42]:
Users_np_array.take(10)


[array([[ 0.36191441,  0.59740863,  0.39422331]]),
 array([[ 0.48353406,  0.2622264 ,  0.4679239 ]]),
 array([[ 0.47093879,  0.46506851,  0.40839008]]),
 array([[ 0.29382593,  0.35745013,  0.33334899]]),
 array([[ 0.4755228 ,  0.44462594,  0.59276764]]),
 array([[ 0.44832381,  0.62101961,  0.45844136]]),
 array([[ 0.56582507,  0.46643984,  0.53454972]]),
 array([[ 0.64450901,  0.75610287,  0.14718829]]),
 array([[ 0.18077385,  0.49655242,  0.60898602]]),
 array([[ 0.52590104,  0.50620125,  0.41785264]])]

In [43]:
nrow=Users_np_array.count()

nrow


671

In [44]:
Items_broadcast = sc.broadcast({
  k: v for (k, v) in Items.collect()
})

In [63]:
userId='404'


In [64]:
k=Users_broadcast.value[userId][0]
k

array([ 0.52909326,  0.37394998,  0.4569707 ])

In [65]:
userRow=Users_broadcast.value[userId][0]
userRow

array([ 0.52909326,  0.37394998,  0.4569707 ])

In [66]:
item_keys= Items.map(lambda x: (x[0])).collect()
item_keys

['90647',
 '7560',
 '38061',
 '66798',
 '94919',
 '66934',
 '2265',
 '118702',
 '3445',
 '3076',
 '4515',
 '5023',
 '3690',
 '145',
 '72393',
 '7706',
 '2232',
 '1281',
 '25868',
 '4334',
 '6591',
 '25773',
 '852',
 '26430',
 '25827',
 '2797',
 '8527',
 '39398',
 '8795',
 '73344',
 '7697',
 '48972',
 '118326',
 '92756',
 '5618',
 '78174',
 '7007',
 '499',
 '3165',
 '114935',
 '93838',
 '37277',
 '7376',
 '3188',
 '27704',
 '85412',
 '96563',
 '110127',
 '7981',
 '50601',
 '78499',
 '3665',
 '7125',
 '2525',
 '1129',
 '834',
 '34437',
 '4903',
 '780',
 '2747',
 '110611',
 '2457',
 '3590',
 '26788',
 '3624',
 '4623',
 '103502',
 '3173',
 '5830',
 '203',
 '5882',
 '4326',
 '2053',
 '108949',
 '50641',
 '1232',
 '89864',
 '61250',
 '4745',
 '1180',
 '4203',
 '67408',
 '2981',
 '25900',
 '1597',
 '3699',
 '43908',
 '92507',
 '75803',
 '3053',
 '73168',
 '72395',
 '2152',
 '1752',
 '106332',
 '2782',
 '2019',
 '27186',
 '1297',
 '113573',
 '38798',
 '6850',
 '2043',
 '7919',
 '8946',
 '34528

In [67]:
int(item_keys[0])

90647

In [68]:
dim=item_user_ratings.count()
dim

9066

In [69]:
u=[0.00000]*9066
user_pred=np.array(u)
user_pred

array([ 0.,  0.,  0., ...,  0.,  0.,  0.])

In [70]:
  #  Userssquare = np.zeros([n_factors.value,n_factors.value])
import itertools
#for i, j in itertools.product(range(x), range(y)):


In [71]:
#for i in range(dim):
#for i,j in itertools.product(range(dim),item_keys):
for i, j in zip(range(dim),item_keys):
    # Stuff...
    itemRow=Items_broadcast.value[j]
    user_pred[i]=(np.dot(k,itemRow.T))
    


In [72]:
for i in range(dim):
    print(user_pred[i])


2.97219710957
3.56740107116
3.98132226846
3.11977540026
2.54301258724
3.98989094332
2.02036229296
2.54301258724
2.98823276069
3.14060005488
3.38118299142
3.89682116162
2.04951331721
3.40825017466
3.71338959587
3.75857340569
4.51696061321
4.22217112857
3.63230564762
3.87564020481
3.87177450564
3.69542412946
3.63099248412
0.923856032364
3.47441811603
4.02353327343
2.66027172447
2.7571930341
2.28091590121
4.49665626544
3.2812030836
1.06913776243
2.03441006979
2.30964008091
4.01571750519
2.83868724382
3.77833741241
2.34678105898
4.07493126238
3.49287412815
4.31477015783
0.923856032364
2.88802582322
3.50111900959
2.32105925405
3.60830159199
3.88000598847
2.6705788503
4.29187917394
3.05944768493
4.38932782305
3.59795982841
3.23349611327
2.63218793825
3.59578581458
3.1478709779
3.89887168912
4.3145576972
3.65751409256
3.17744598514
2.54301258724
3.20626447953
3.28743519554
3.1391880148
3.62289018126
4.39213073718
1.52580755234
3.36307727421
3.0181458128
3.43225325081
3.10196758931
4.254654028

In [73]:
# The item matrix is needed in all partitions when solving for rows of User matrix individually
movies_broadcast = sc.broadcast({
  k: v for (k, v) in movies_data.collect()
})

In [74]:
movies_broadcast.value['90647']

'Puss in Boots (2011)'

In [78]:
top_pred_keys=user_pred.argsort()[-5:][::-1]
top_pred_keys


array([ 294, 1632, 4586, 4533, 4127], dtype=int64)

In [77]:
top_movie_keys=np.array([0]*5)
print("Recommended movies for userId", userId)
for i, j in zip(range(5),top_pred_keys):
    print("Movie Id=", item_keys[j], " with title=", movies_broadcast.value[item_keys[j]])

Recommended movies for userId 404
Movie Id= 1563  with title= Dream With the Fishes (1997)
Movie Id= 4076  with title= Two Ninas (1999)
Movie Id= 4930  with title= Funeral in Berlin (1966)
Movie Id= 4796  with title= "Grass Is Greener
Movie Id= 1819  with title= Storefront Hitchcock (1997)


In [ ]:
###################################CONTENT BASED SIMILARITY ############################


In [79]:
from numpy import linalg as LA

In [80]:
def cosineSimilarity(vec1, vec2):
  return vec1.dot(vec2) / (LA.norm(vec1) * LA.norm(vec2))

In [98]:
user_ratings=ratings_data.map(lambda x: (x[0], x[2]))

In [97]:
user_ratings.take(1)

[('190', <pyspark.resultiterable.ResultIterable at 0x1bbe0b0d6a0>)]

In [83]:
ur_broadcast = sc.broadcast({
  k: v for (k, v) in user_ratings.collect()
})

In [88]:
ur_broadcast.value['250']

'5.0'

In [93]:
user_cos=ur_broadcast.value[userId][0]
user_cos

'4'

In [141]:
user_ratings=ratings_data.map(lambda X: (X[0], X[2]))

In [246]:
movie=ratings_data.map(lambda x: (x[2],x[1]))
movie.take(3)

[('2.5', '31'), ('3.0', '1029'), ('3.0', '1061')]

In [242]:
u_r= np.array(user_ratings.lookup('20'))
#array1= (u_r.argsort()[-5:][::1])


In [243]:
array1=u_r[u_r.argsort()[-5:]]
str(array1[0])

'5.0'

In [ ]:
 SSE = user_item_ratings.map(getRowSumSquares).reduce(lambda a, b: a + b)
  Count = ratings_data.count()
  MSE = SSE / Count
  print ("MSE:", MSE)

In [237]:
m_broadcast = sc.broadcast({
  k: v for (k,v) in movie.collect()
})

In [245]:
top_movie=m_broadcast.value['5']
top_movie

'5952'

In [189]:
g_m=movies_raw_data.filter(lambda line: line!=movies_raw_data_header)\
    .map(lambda line: line.split(",")).map(lambda tokens: (tokens[0],tokens[2])).cache()

In [212]:
m_g=movies_raw_data.filter(lambda line: line!=movies_raw_data_header)\
    .map(lambda line: line.split(",")).map(lambda tokens: (tokens[2],tokens[0])).cache()

In [190]:
g_m.take(3)

[('1', 'Adventure|Animation|Children|Comedy|Fantasy'),
 ('2', 'Adventure|Children|Fantasy'),
 ('3', 'Comedy|Romance')]

In [198]:
m_broadcast = sc.broadcast({
  k: v for (k,v) in g_m.collect()
})

In [203]:
top_genre=m_broadcast.value[top_movie]

In [204]:
g_broadcast = sc.broadcast({
  v: k for (k,v) in g_m.collect()
})

In [221]:
m_r_broadcast = sc.broadcast({
  v: k for (k,v) in movie.collect()
})

In [224]:
lookup_content=np.array(m_g.lookup(top_genre))
lookup_content

array(['5065', '5171', '5267', '5296', '5297', '5329', '5363', '5380',
       '5400', '5418', '5421', '5460', '5462', '5483', '5501', '5505',
       '5523', '5574', '5611', '5619', '5621', '5666', '5679', '5787',
       '5876', '5942', '5952', '5958', '5992', '5995', '6060', '6207',
       '6279', '6335', '6342', '6591', '6592', '7266', '8011'], 
      dtype='<U4')

In [218]:
genre_ratings=[0.0000]*len(lookup_content)

In [222]:
for i, j in zip(range(5),lookup_content):
    genre_ratings[i]=m_r_broadcast.value[j]

In [228]:
g_r=np.array(genre_ratings)
g_r

array(['2.5', '4.0', '4.0', '3.0', '4.0', '0.0', '0.0', '0.0', '0.0',
       '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0',
       '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0',
       '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0',
       '0.0', '0.0', '0.0'], 
      dtype='<U3')

In [229]:
top_movies_genre=lookup_content[g_r.argsort()[-5:]]
top_movies_genre

array(['5065', '5296', '5297', '5267', '5171'], 
      dtype='<U4')

In [230]:
print("Recommended movies are:" )


Recommended movies are:


In [232]:
print("Recommended movies for userId", userId)
for i, j in zip(range(5),top_movies_genre):
    print("Movie Id=", j, " with title=", movies_broadcast.value[top_movies_genre[i]])

Recommended movies for userId 404
Movie Id= 5065  with title= "Mothman Prophecies
Movie Id= 5296  with title= "Sweetest Thing
Movie Id= 5297  with title= "Cat's Meow
Movie Id= 5267  with title= "Rookie
Movie Id= 5171  with title= "Time Machine
